### Exercises

**Q1**: Try to build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set. Hint: the KNeighborsClassifier works quite well for this task; you just need to find good hyperparameter values (try a grid search on the weights and n_neighbors hyperparameters).

**A1**:

In [11]:
# get the data
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home='./tmp')

In [12]:
X, y = mnist["data"], mnist["target"]
print(mnist.DESCR)
print(X.shape)
print(y.shape)

mldata.org dataset: mnist-original
(70000, 784)
(70000,)


The MINST dataset is already split into training (60,000) and test (10,000) entries, so let's separate those.

Also, the data is organized by digit, but we should randomize the order as some algorithms are sensitive to pre-sorted data:

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

import numpy as np

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


knn_clf = KNeighborsClassifier(n_jobs=4)
param_grid = [
    {'weights': ['uniform', 'distance'], 'n_neighbors': [1,10,100]}
  ]

knn_grid_search = GridSearchCV(knn_clf, param_grid, cv=3,
                           scoring='accuracy', n_jobs=4, verbose=3)

knn_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] n_neighbors=1, weights=uniform ..................................
[CV] n_neighbors=1, weights=uniform ..................................
[CV] n_neighbors=1, weights=uniform ..................................
[CV] n_neighbors=1, weights=distance .................................
